In [1]:
import sys
sys.path.append("/home/admin/blessed/HandWritten_Amharic_English_OCR")
from dataset.dataloader import MyOcrDataloader
import pandas as pd
import math
import random
import torch
import yaml
import os
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from utils.utils import forward_pass_with_labels, update
from jiwer import wer, cer

from tqdm.auto import tqdm

2025-01-16 16:24:32.189351: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-16 16:24:32.230299: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-16 16:24:32.928235: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
config_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), '..')), "config/main.yaml")
with open(config_path, "r") as file:
    config = yaml.safe_load(file)
DATASET_PATH =config["DATASET_PATH"]
MODEL_ID = config["MODEL_ID"]

device = "cuda" if torch.cuda.is_available() else "cpu"
processor = TrOCRProcessor.from_pretrained(MODEL_ID)
model =VisionEncoderDecoderModel.from_pretrained(MODEL_ID).to(device)
model.config.decoder_start_token_id = processor.tokenizer.eos_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
## sanity check for data loader
os.chdir("../dataset")
data = MyOcrDataloader(DATASET_PATH, preprocessor=processor)
train_loader = torch.utils.data.DataLoader(
    dataset     = data,
    batch_size  = config['BATCH_SIZE'],

    shuffle     = True,
    collate_fn= data.collate_fn
    )

for batch in train_loader:
    print(batch[0].shape, batch[1].shape,processor.tokenizer.batch_decode(batch[1], skip_special_tokens=True))
    
    break

torch.Size([8, 3, 384, 384]) torch.Size([8, 7]) ['piassa', '8', '05/22/1990', '16/12/1988', '2', '1990/02/06', '1000', '8']


In [7]:
def one_train_epoch(model, train_data_loader, scaler, optimizer):
    model.train()
    pbar = tqdm(range(len(train_data_loader)))
    train_loss = 0
    train_wers = 0.0
    train_cers = 0.0
    samples = 0
    for i,batch in enumerate(train_data_loader):
        loss, logits = update(model, scaler, optimizer, batch)
        preds = processor.batch_decode(logits.argmax(dim=2), skip_special_tokens=True)
        actuals = processor.batch_decode(batch[1], skip_special_tokens=True)

        train_loss += (1/(i+1))*(loss-train_loss)
        # print(logits.shape,"preds", preds, "\n", "="*20, "\nactuals", actuals)
        for pred, actual in zip(preds, actuals):
            train_wers += wer(actual, pred)
            train_cers += cer(actual, pred)

        samples += len(preds)
   
        pbar.set_description(f"Train Loss: {train_loss:.4f} WER: {train_wers/samples:.4f} CER: {train_cers/samples:.4f}")
        pbar.update(1)
    print("pred", preds[:2])
    print("actual", actuals[:2])
    pbar.close()
    return train_loss, train_wers/samples, train_cers/samples

In [8]:
def train(model, tdataloader,vdataloader, optimizer, scaler, epochs=3, log_dir="logs/a"):
    os.makedirs(log_dir, exist_ok=True)
    with open(f"{log_dir}/train.txt", "w") as tf, open(f"{log_dir}/val.txt", "w") as vf:
        tf.write(f"Epoch,Loss,Score\n")
        vf.write("Epoch,Loss,Score\n")
    best_val_score = 0    
    train_loss = 0
    for epoch in range(1, 1+epochs):
        loss, wers, cers  = one_train_epoch(model, tdataloader, optimizer, scaler )
        # with open(f"{log_dir}/train.txt", "a+") as tf, open(f"{log_dir}/val.txt", "a+") as vf:
        #     tf.write(f"{epoch},{train_loss},{tscore}\n")
        #     vf.write(f"{epoch},{val_loss},{score}\n")

In [11]:

optimizer = torch.optim.AdamW(model.parameters(), 1e-4)
optimizer.zero_grad()




scaler =  torch.cuda.amp.GradScaler(enabled=True)

train(model, train_loader, train_loader,optimizer, scaler, epochs=15,log_dir="logs/phi_teleqna")

/tmp/ipykernel_2654980/455183555.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler =  torch.cuda.amp.GradScaler(enabled=True)


  0%|          | 0/43 [00:00<?, ?it/s]

pred ['', '']
actual ['11/03/1989', '30']


  0%|          | 0/43 [00:00<?, ?it/s]

pred ['', '']
actual ['5', 'Addis Ababa']


...

pred ['', '']
actual ['-', '4']


  0%|          | 0/43 [00:00<?, ?it/s]

pred ['', '']
actual ['Addis Ababa', '27/04/84']


  0%|          | 0/43 [00:00<?, ?it/s]

KeyboardInterrupt: 